In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/normal/ia_500000023.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save('second_try.h5') 

Found 572 images belonging to 2 classes.
Found 117 images belonging to 2 classes.
Epoch 1/20
125/125 [==============================] - 25s 202ms/step - loss: 0.6698 - accuracy: 0.6205 - val_loss: 0.5439 - val_accuracy: 0.7343
Epoch 2/20
125/125 [==============================] - 25s 196ms/step - loss: 0.5920 - accuracy: 0.7108 - val_loss: 0.4752 - val_accuracy: 0.7425
Epoch 3/20
125/125 [==============================] - 27s 213ms/step - loss: 0.5654 - accuracy: 0.7289 - val_loss: 0.5673 - val_accuracy: 0.7888
Epoch 4/20
125/125 [==============================] - 25s 204ms/step - loss: 0.5357 - accuracy: 0.7389 - val_loss: 0.4706 - val_accuracy: 0.7939
Epoch 5/20
125/125 [==============================] - 27s 215ms/step - loss: 0.5072 - accuracy: 0.7626 - val_loss: 0.3333 - val_accuracy: 0.8460
Epoch 6/20
125/125 [==============================] - 25s 202ms/step - loss: 0.4807 - accuracy: 0.7862 - val_loss: 0.3647 - val_accuracy: 0.8692
Epoch 7/20
125/125 [============================

In [40]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import load_model


model = load_model("second_try.h5")

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


imagePath = "predict_dir/normal/ia_100003833.jpg"
test_image = image.load_img(imagePath, target_size = (150, 150)) 
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = model.predict(test_image)

print(result[0])

[0.]


In [43]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import load_model


model = load_model("second_try.h5")

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


imagePath = "predict_dir/normal/ia_100003834.jpeg"
img = image.load_img(imagePath, target_size = (150, 150)) 
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict_classes(images, batch_size=4)
print (classes)

# predicting multiple images at once
img = image.load_img('predict_dir/normal/ia_100003836.jpg', target_size=(150, 150))
y = image.img_to_array(img)
y = np.expand_dims(y, axis=0)

# pass the list of multiple images np.vstack()
images = np.vstack([x, y])
classes = model.predict_classes(images, batch_size=4)

# print the classes, the images belong to
print (classes)
print (classes[0])
print (classes[0][0])



[[0]]
[[0]
 [0]]
[0]
0
